---이 노트북은 [여기](https://colab.research.google.com/github/google-research/google-research/blob/master/flow_matching/flow_matching_tutorial.ipynb)에서 찾을 수 있는 Colab 노트북의 JAX/Flax 버전입니다.원저자: Yaron Lipman, Ricky T. Q. Chen, Heli Ben-Hamu.---# Flow Matching: 이미지 생성 튜토리얼이 노트북은 Flow Matching(Lipman et al., 2022)을 사용하여 간단한 2D 데이터셋에 대한 생성 모델을 훈련시키는 방법을 보여줍니다. Flow Matching은 최근에 제안된 방법으로, 안정적이고 효율적인 방식으로 연속 정규화 흐름(CNF)을 훈련시키는 방법입니다.이 노트북은 다음 내용을 다룹니다:1. Flow Matching을 사용하여 CNF를 훈련시키는 방법에 대한 간략한 개요2. 데이터 로딩 및 사전 처리3. Flow Matching을 사용하여 신경망(U-Net)을 훈련시켜 벡터 필드를 학습하는 방법4. 훈련된 모델을 사용하여 새로운 이미지를 생성하는 방법5. 훈련된 모델의 확률 밀도를 평가하는 방법

## 1. Flow Matching으로 연속 정규화 흐름 훈련하기연속 정규화 흐름(CNF)은 상미분 방정식(ODE)의 해인 함수 $phi_t$의 매개변수화된 족입니다. 관례적으로, CNF는 시간 $t=0$에서 시작하여 시간 $t=1$에서 끝납니다. CNF는 매끄러운 가역 함수이므로, 간단한 분포(예: 표준 정규 분포)를 복잡한 분포(예: 이미지)로 변환하는 데 사용할 수 있습니다. 이는 다음과 같이 달성됩니다:1. 알려진 사전 분포 $p_0$에서 샘플 $x_0$을 샘플링합니다.2. ODE $\frac{d\phi_t(x)}{dt} = v_t(\phi_t(x))$를 초기 조건 $\phi_0(x) = x$로 시간 $[0,1]$ 동안 풀어 $x_1 = \phi_1(x_0)$을 얻습니다.결과 $x_1$은 $p_1$ 분포를 따르는 샘플입니다. 여기서 $p_1 = (\phi_1)_* p_0$은 $p_0$을 $\phi_1$으로 푸시포워드한 것입니다.우리는 $x_0 \sim p_0$을 샘플링하고 $x_1 = \phi_1(x_0)$을 계산하여 $p_1$에서 샘플을 생성할 수 있습니다.Flow Matching의 목표는 주어진 데이터 분포 $p_1$을 푸시포워드 분포 $(\phi_1)_* p_0$과 일치시키는 것입니다. 이는 시간 종속 벡터 필드 $v_t$를 매개변수화하는 신경망을 훈련시켜 달성됩니다. Flow Matching은 $p_t(x)$를 $p_0$과 $p_1$ 사이의 보간으로 정의하고, 이러한 경로를 따라 점을 이동시키는 벡터 필드 $u_t(x)$를 정의합니다. 그런 다음 단순히 $v_t(x)$가 $u_t(x)$와 일치하도록 회귀 손실을 최소화합니다.Flow Matching의 핵심 아이디어는 다음과 같습니다:1. 시간 $t \in [0, 1]$을 샘플링합니다.2. $p_t(x)$에서 $x_t$를 샘플링합니다.3. $x_t$에서 목표 벡터 필드 $u_t(x_t)$를 계산합니다.4. 손실 $L = \|v_t(x_t) - u_t(x_t)\|^2$를 최소화합니다.이 프로세스는 아래 그림에 요약되어 있습니다.![Flow Matching](https://raw.githubusercontent.com/google-research/google-research/master/flow_matching/images/flow_matching.png)

## 2. 설정

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from tqdm import trange

import flax.linen as nn
from flax.training import train_state
import optax
from diffrax import diffeqsolve, ODETerm, Dopri5, SaveAt, Tsit5

import matplotlib.pyplot as plt

### 데이터 로딩

이 노트북에서는 간단한 2D 데이터셋을 사용합니다. 데이터셋은 두 개의 달이 있는 이미지입니다. 우리는 이 이미지를 다운로드하여 NumPy 배열로 변환합니다. 그런 다음 데이터셋을 정규화하고 훈련, 테스트 및 검증 세트로 분할합니다.

In [ ]:
def get_dataset(img_size=64):
  """두 개의 달 데이터셋을 다운로드하고 사전 처리합니다."""
  # 이미지 다운로드
  url = 'https://raw.githubusercontent.com/google-research/google-research/master/flow_matching/images/two_moons.png'
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))

  # 이미지를 NumPy 배열로 변환
  img = img.resize((img_size, img_size), Image.Resampling.LANCZOS)
  img = np.array(img, dtype=np.float32) / 255.0
  # 검은색 픽셀 찾기
  coords = np.stack(np.where(img.mean(-1) < 0.5), 1).astype(np.float32)

  # 데이터 정규화
  coords /= (img_size - 1)
  coords = coords * 2 - 1

  # 훈련, 테스트 및 검증 세트로 분할
  np.random.shuffle(coords)
  train_size = int(0.8 * len(coords))
  val_size = int(0.1 * len(coords))

  train_data = coords[:train_size]
  val_data = coords[train_size:train_size+val_size]
  test_data = coords[train_size+val_size:]

  return train_data, val_data, test_data

img_size = 32
train_data, val_data, test_data = get_dataset(img_size=img_size)

# 데이터셋 시각화
plt.figure(figsize=(5, 5))
plt.scatter(train_data[:, 0], train_data[:, 1], s=1)
plt.title('Two Moons Dataset')
plt.show()

## 3. 모델 정의

우리는 벡터 필드 $v_t(x)$를 매개변수화하기 위해 U-Net 아키텍처를 사용합니다. U-Net은 이미지 분할 작업에 일반적으로 사용되는 신경망 유형입니다. 이는 인코더와 디코더로 구성됩니다. 인코더는 입력 이미지를 저차원 표현으로 다운샘플링하고, 디코더는 이 표현을 원래 이미지 크기로 업샘플링합니다. U-Net은 또한 인코더와 디코더 사이에 스킵 연결을 가지고 있어, 디코더가 인코더의 특징에 접근할 수 있도록 합니다. 이는 U-Net이 이미지의 전역 및 로컬 특징을 모두 학습하는 데 도움이 됩니다.

우리의 경우 입력은 $(x, t)$ 쌍이고, 출력은 벡터 필드 $v_t(x)$입니다. 여기서 $x$는 데이터 포인트(좌표)이고 $t$는 시간입니다. 우리는 시간 $t$를 네트워크에 주입하기 위해 정규화 흐름에서 사용되는 표준 기술인 가우시안 푸리에 특징을 사용합니다.

In [ ]:
class GaussianFourierProjection(nn.Module):
    """가우시안 푸리에 특징 시간 임베딩."""
    embedding_size: int = 256
    scale: float = 1.0

    @nn.compact
    def __call__(self, x):
        W = self.param('W', jax.nn.initializers.normal(stddev=self.scale), [self.embedding_size])
        W = jax.lax.stop_gradient(W)
        x_proj = x[:, None] * W[None, :] * 2 * jnp.pi
        return jnp.concatenate([jnp.sin(x_proj), jnp.cos(x_proj)], axis=-1)

class Dense(nn.Module):
    """시간 임베딩을 사용하는 완전 연결 레이어."""
    features: int
    
    @nn.compact
    def __call__(self, x, t):
        y = nn.Dense(features=self.features)(x)
        y += nn.Dense(features=self.features, use_bias=False)(t)
        return y


class UNet(nn.Module):
    """간단한 U-Net 아키텍처."""
    
    @nn.compact
    def __call__(self, x, t):
        # 시간 임베딩
        t_embedding = GaussianFourierProjection()(t.ravel())
        t_embedding = nn.Dense(features=x.shape[-1])(t_embedding)
        t_embedding = nn.silu(t_embedding)
        t_embedding = nn.Dense(features=x.shape[-1])(t_embedding)

        # U-Net
        x = nn.Dense(features=64)(x)
        h1 = nn.relu(x)

        x = Dense(features=128)(h1, t_embedding)
        h2 = nn.relu(x)

        x = Dense(features=256)(h2, t_embedding)
        h3 = nn.relu(x)

        x = Dense(features=128)(h3, t_embedding)
        x = nn.relu(x)
        x = jnp.concatenate([x, h2], axis=-1)

        x = Dense(features=64)(x, t_embedding)
        x = nn.relu(x)
        x = jnp.concatenate([x, h1], axis=-1)

        x = nn.Dense(features=x.shape[-1])(x)
        return x

## 4. 훈련

우리는 Flow Matching을 사용하여 모델을 훈련시킵니다. Flow Matching 손실은 다음과 같이 주어집니다:
$$L(\theta) = E_{t, p_t(x)}[||v_\theta(x, t) - u_t(x)||^2]$$

여기서 $v_\theta$는 우리의 신경망이고, $u_t$는 목표 벡터 필드입니다. 조건부 흐름 매칭을 사용하며, 여기서 $p_t(x) = p_t(x|x_1)$는 $x_1 \sim p_{data}$가 주어졌을 때 $x$에 대한 조건부 확률 경로입니다. 이 경우, 우리는 다음을 선택합니다:
$$p_t(x|x_1) = N(x; (1-t)x_0 + tx_1, \sigma^2)$$
$$u_t(x|x_1) = x_1 - x_0$$

우리는 $x_0$을 표준 정규 분포에서 샘플링합니다. 이 선택은 Lipman et al., 2022의 "조건부 흐름 매칭"에 해당합니다.

In [ ]:
def loss_fn(params, state, batch, key):
    """Flow Matching 손실 함수."""
    x1 = batch
    key, subkey = jax.random.split(key)
    x0 = jax.random.normal(subkey, shape=x1.shape)
    key, subkey = jax.random.split(key)
    t = jax.random.uniform(subkey, shape=(x1.shape[0], 1))

    xt = (1 - t) * x0 + t * x1
    ut = x1 - x0

    vt = state.apply_fn({'params': params}, xt, t)
    loss = jnp.mean((vt - ut)**2)
    return loss

def get_train_step(optimizer):
    """단일 훈련 단계를 반환합니다."""
    @jax.jit
    def train_step(state, batch, key):
        grad_fn = jax.value_and_grad(loss_fn)
        loss, grads = grad_fn(state.params, state, batch, key)
        state = state.apply_gradients(grads=grads)
        return state, loss
    return train_step

def train_model(train_data, val_data, num_epochs=1000, batch_size=128):
    """모델을 훈련시킵니다."""
    # 모델 및 옵티마이저 초기화
    model = UNet()
    key = jax.random.PRNGKey(0)
    key, subkey = jax.random.split(key)
    params = model.init(subkey, jnp.zeros((1, 2)), jnp.zeros((1, 1)))['params']
    optimizer = optax.adam(1e-3)
    state = train_state.TrainState.create(apply_fn=model.apply, params=params, tx=optimizer)

    # 훈련 단계 가져오기
    train_step = get_train_step(optimizer)

    # 훈련 루프
    losses = []
    val_losses = []
    for epoch in trange(num_epochs):
        # 훈련 데이터 셔플
        key, subkey = jax.random.split(key)
        perms = jax.random.permutation(subkey, len(train_data))
        train_data = train_data[perms]

        # 배치 반복
        for i in range(0, len(train_data), batch_size):
            batch = train_data[i:i+batch_size]
            key, subkey = jax.random.split(key)
            state, loss = train_step(state, batch, subkey)
            losses.append(loss)

        # 검증 손실 계산
        key, subkey = jax.random.split(key)
        val_loss = loss_fn(state.params, state, val_data, subkey)
        val_losses.append(val_loss)

        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')
    
    return state, losses, val_losses

state, losses, val_losses = train_model(train_data, val_data)

### 손실 곡선 플로팅

In [ ]:
plt.plot(losses, label='Train Loss')
plt.plot(np.arange(len(val_losses)) * (len(losses) // len(val_losses)), val_losses, label='Validation Loss')
plt.yscale('log')
plt.legend()
plt.title('Loss Curves')
plt.show()

## 5. 샘플링

이제 모델이 훈련되었으므로, 이를 사용하여 새로운 샘플을 생성할 수 있습니다. 이는 다음 단계를 통해 수행됩니다:
1. 사전 분포 $p_0$에서 샘플 $x_0$을 샘플링합니다.
2. ODE $\frac{d\phi_t(x)}{dt} = v_t(\phi_t(x))$를 초기 조건 $\phi_0(x) = x$로 시간 $[0,1]$ 동안 풀어 $x_1 = \phi_1(x_0)$을 얻습니다.

우리는 `diffrax` 라이브러리를 사용하여 ODE를 풉니다. `diffrax`는 JAX에서 미분 방정식을 풀기 위한 라이브러리입니다. 이는 다양한 솔버를 제공하며, 우리는 Tsit5 솔버를 사용합니다.

In [ ]:
def sample(state, key, num_samples=1000):
    """훈련된 모델에서 샘플을 생성합니다."""
    # ODE 함수 정의
    def ode_func(t, y, args):
        return state.apply_fn({'params': state.params}, y, jnp.array([t]))
    
    term = ODETerm(ode_func)
    solver = Tsit5()
    t0, t1 = 0, 1
    dt0 = 0.1
    saveat = SaveAt(ts=[t1])

    # 사전 분포에서 샘플링
    key, subkey = jax.random.split(key)
    x0 = jax.random.normal(subkey, shape=(num_samples, 2))

    # ODE 풀기
    sol = diffeqsolve(term, solver, t0, t1, dt0, x0, saveat=saveat)
    return sol.ys[0]

key = jax.random.PRNGKey(42)
samples = sample(state, key, num_samples=1000)

### 생성된 샘플 플로팅

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(samples[:, 0], samples[:, 1], s=1, label='Generated Samples')
plt.scatter(train_data[:, 0], train_data[:, 1], s=1, label='Train Data', alpha=0.1)
plt.legend()
plt.show()

## 6. 결론

이 노트북에서는 Flow Matching을 사용하여 간단한 2D 데이터셋에 대한 생성 모델을 훈련시키는 방법을 보여주었습니다. 우리는 Flow Matching을 사용하여 CNF를 훈련시키는 방법, 데이터 로딩 및 사전 처리 방법, Flow Matching을 사용하여 신경망(U-Net)을 훈련시켜 벡터 필드를 학습하는 방법, 훈련된 모델을 사용하여 새로운 이미지를 생성하는 방법을 다루었습니다.